# Multi-speaker speech synthesis

В этом задании мы создадим датасет для синтеза речи и обучим на нем мультиспикерную модель.

Мы уже написали/скопипастили в этот ноутбук много релевантного кода, большая часть из которого взята из [репозитория FastSpeech2](https://github.com/ming024/FastSpeech2). Вам придется:

- запускать этот код, визуализировать данные и принимать решения - как разбить данные, какое распределение должно быть у фичей

- запускать инференс обученной модели

Некоторые части кода нужно будет все-таки заполнить - это поможет лучше разобраться в том, как получаются фичи и как учится модель синтеза. Рекомендуем разобраться в коде внутри этого ноутбука. Также можно смотреть и на другие файлы этого репозитория.

### Основные части задания:
1. Создание датасета (2 балла). Мы нашли для вас несколько видео с речью знаменитостей - но это длинные дорожки по полчаса. Нужно сделать из них пригодные для обучения данные.
2. Генерация фичей (2 балла). Извлекаем информацию про интонацию и скорость речи.
3. Обучение модели синтеза (3 балла). На это может уйти до 10 часов - не оставляйте задание на последний вечер!
4. Анализ результатов (8 баллов): меняем интонацию, смотрим на ошибки модели.

### Про модель синтеза

![alt text](synthesized_melspectrogram.png "Mel Spectrogram")

В качестве модели синтеза мы будем учить [FastSpeech2](https://arxiv.org/abs/2006.04558). На выход она выдает [мел-спектрограмму](https://medium.com/analytics-vidhya/understanding-the-mel-spectrogram-fca2afa2ce53). Напомним, как она строится:

- дана звуковая волна размерности `sample_rate` значений в секунду
- она разбивается на промежутки размера `window_length`
- левые границы промежутков находятся на расстоянии `hop_length` друг от друга
- по каждому промежутку считается FFT, после логарифмирования и перевода в mel-базис получается вектор (будем называть его фреймом)
- вектора конкатенируются, таким образом по горизонтальной оси в мел-спеке идет время.

Преобразованием мел-спеки в аудио занимается отдельная модель - вокодер. В этом ноутбуке мы используем предобученный [MelGAN](https://arxiv.org/abs/1910.06711).

#### Входы модели


1. список фонем

2. длины всех фонем (alignment). Мы считаем, что каждой фонеме соответствует целое число фреймов.

3. интонационные фичи ([f0](https://wiki.aalto.fi/pages/viewpage.action?pageId=149890776) и energy), посчитанные для каждого фрейма

4. эмбеддинг спикера

Пункты (2) и (3) наша модель будет уметь предсказывать. Но примеры из валидационного датасета можно будет синтезировать как с предсказанными, так и с ground truth фичами.


# 0. Готовим данные и окружение

In [ ]:
!git clone https://github.com/andrewgolman/YSDA_PracticalDS_Synthesis.git
# for colab
%cd YSDA_PracticalDS_Synthesis
!bash setup.sh

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import shutil
from pathlib import Path
from tqdm.notebook import tqdm
import numpy as np

from IPython.display import Audio
import librosa
from omegaconf import OmegaConf

In [ ]:
data_path = Path("ysda_books_data")
base_path = Path("dataset/")
vctk_path = Path("vctk_data/")
!mkdir {base_path}

In [ ]:
# # # books and celebs data
!wget https://intone-public-data.s3.us-east-2.amazonaws.com/ysda_books_data.zip
!unzip ysda_books_data.zip -d {data_path}
!mv {data_path}/shad_data/* {data_path}

# # # features from VCTK
!wget https://intone-public-data.s3.us-east-2.amazonaws.com/vctk_features.zip
!unzip vctk_features.zip -d {vctk_path}
!mv {vctk_path}/vctk_base_256/* {vctk_path}

# # # audio from VCTK
!wget https://intone-public-data.s3.us-east-2.amazonaws.com/vctk_audio.zip 
!unzip vctk_audio.zip -d vctk_audio

In [ ]:
data_cfg = OmegaConf.load("conf/data.yaml")

#  1. Подготовка датасета

### 1.1 Разбиение данных
Мы будем обучаться на двух датасетах.

1. VCTK - популярный в рисерче датасет для мультиспикерного синтеза. Посмотрите как он устроен, послушайте примеры из него - обычно это короткие предложения по 3-7 секунд (что узнать точнее, можно посмотреть на распределение длин записей).
2. Датасет, создаваемый в этом ноутбуке. Мы собрали несколько аудиокниг, а также записей селебрити - вы можете также добавить в датасет любые свои записи, включая собственный голос. Также нужно добавить текст, соответствующий записям. (см. формат в директории `data_path`)

Но на 30-минутных записях не обучишься - давайте разобьем их на более короткие. Разбивать будем по участкам с тишиной - низкой энергией.

Подберите параметры для такого сплита.

In [ ]:
from pydub import AudioSegment
import auditok

split_path = base_path / "split"

split_args = {
    "min_dur": <...>,
    "max_dur": <...>,
    "max_silence": <...>,
    "energy_threshold": <...>,
    # put other options you need here
}

def split(wav_path, output_dir, meta, split_args):
    os.makedirs(output_dir, exist_ok=True)
    
    wav = AudioSegment.from_wav(wav_path)
    audio_regions = auditok.split(str(wav_path), **split_args)
    
    duration_in_seconds = 0
    regions_count = 0
    
    for i, r in enumerate(audio_regions):
        start = r.meta.start * 1000
        end = r.meta.end * 1000
        start = <...> # do you need any other adjustment?
        end = <...>
        duration_in_seconds += r.meta.end - r.meta.start
        wav[start:end].export(
            output_dir / f"{meta}_{int(r.meta.start)}s.wav",
            format="wav"
        )
    print(f"Found {i} regions in track {wav_path} of duration {duration_in_seconds / 60} minutes")


In [ ]:
if os.path.exists(split_path):
    shutil.rmtree(split_path)  # clean up before each run to avoid looking at files from previous split


for filename in os.listdir(data_path):
    if filename.endswith(".wav"):
        speaker = filename.split("_")[0]
        split(
            wav_path=data_path / filename,
            output_dir=split_path / speaker,
            meta=filename[:-4],
            split_args=split_args
        )


In [ ]:
# here is how dataset looks
!ls {split_path}/obama | head

## 1.2 Генерация текстов

В идеальном мире любые данные можно быстро разметить краудсорсингом. Но в реальности краудсорсинг медленный и недешевый, поэтому часть данных нужно стараться разметить автоматически. Мы реализуем такую разметку, а все записи в которых сомневаемся - удалим (оставшихся данных должно хватить).

Шаг 1. Найти тексты к нашим записям и считать их.

Шаг 2. Применить к нашим записям ASR.

Шаг 3. Найти в текстах записи, соответствуеющие выходам ASR. Если два вида разметки совпадают, разметка близка к истине.

Третий шаг будем делать с помощью библиотеки fuzzysearch - она ищет в большом тексте ближайший совпадающий текстовый отрезок.
Алгоритм предлагаем реализовать такой:
1. Посмотреть на найденную подстроку. Если его границы находятся в середине слова - добавим несколько символом слева и справа, чтобы все слова входили в подстроку полностью.
2. По расстоянию Левенштейна проверить, что полученная строка примерно совпадает с исходной. Если расстояние слишком большое, то не будем включать наш пример в датасет.

Валидируйте параметры матчинга на небольшом наборе записей, чтобы не запускать ASR несколько раз - каждый запуск может занять несколько часов.

In [ ]:
from speechbrain.pretrained import EncoderDecoderASR

asr_model = EncoderDecoderASR.from_hparams(
    source="speechbrain/asr-crdnn-transformerlm-librispeech",
    savedir="pretrained_models/asr-crdnn-transformerlm-librispeech",
    run_opts={'device': 'cuda'}
)

In [ ]:
import fuzzysearch
import Levenshtein

import data
from utils import timeout

def apply_asr(wav_path, text):
    caption = asr_model.transcribe_file(str(wav_path)).lower()
    if not caption:
        return None

    matches = timeout(
        fuzzysearch.find_near_matches, args=[caption, text],
        kwargs={'max_l_dist': <...>}, timeout_duration=3
    )
    if matches:
        # DO NOT CUT MIDDLE OF THE WORD
        l, r = matches[0].start, matches[0].end
        # YOUR CODE HERE
        l = <...>
        r = <...>
        matched_output = text[l:r].strip()

        if Levenshtein.distance(caption, matched_output) < <...>:
            return matched_output

    return None

In [ ]:
# RUN ASR ON THE DATASET

texts = {}
for filename in os.listdir(data_path):
    if filename.endswith(".txt"):
        with open(data_path / filename) as f:
            texts[filename[:-4]] = " ".join(f.readlines())


for spk in os.listdir(split_path):
    print("Speaker:", spk)
    for filename in tqdm(os.listdir(split_path / spk)):
        if not filename.endswith(".wav"):
            continue
        spk = filename.split("_")[0]
        tag = filename.split("_")[1]
        text = texts[f"{spk}_{tag}"]
        filename = split_path / spk / filename
        out_path = Path(filename).with_suffix('.lab')
        caption = apply_asr(filename, text)
        if caption:
            with open(out_path, 'w') as f:
                f.write(caption)

- Посмотрите на оставшиеся и удаленные примеры. Как выбирались пороги ?

- Посчитайте длину оставшихся для обучения данных. Для каждого спикера рекомендуем оставить не менее 15 минут, но модель способна обучиться и на 3 минутах.

In [ ]:
<YOUR CODE HERE>

## 2. Генерация фичей

Наша модель синтеза речи будет предсказывать следующие фичи:

- алайнмент (длины каждой фонемы - целое число фреймов мел-спектрограммы)
- энергию (коррелирует с громкостью) на каждом фрейме.
- f0 (коррелирует с питчом) на каждом фрейме.

При синтезе из текста эти фичи определят стиль и интонацию фразы. Также мы сможем сделать синтез, используя все эти фичи из оригинальной записи - чтобы оценить качество звука.

f0 и энергии мы посчитаем сами - вам нужно будет только их нормализовать, чтобы модели было удобно работать с этими фичами.

А вот алайнмент будем строить вместе. Запустим Montreal Force Aligner, чтобы узнать время начала и конца каждой фонемы.

In [ ]:
!source miniconda/bin/activate && \
    mfa g2p -j8 \
    english_g2p.zip \
    {split_path} {base_path}/phoneme_dict.txt
    
    
!source miniconda/bin/activate && \
    mfa align --clean -j16 \
    {split_path} {base_path}/phoneme_dict.txt \
    english.zip \
    {base_path}/TextGrid

In [ ]:
!ls {base_path}/TextGrid/trump

In [1]:
!cat /Users/andrewgolman/shad_task/tests/alignment.TextGrid

File type = "ooTextFile"
Object class = "TextGrid"

xmin = 0 
xmax = 3.6985 
tiers? <exists> 
size = 2 
item []: 
    item [1]:
        class = "IntervalTier" 
        name = "words" 
        xmin = 0 
        xmax = 3.6985 
        intervals: size = 13 
        intervals [1]:
            xmin = 0 
            xmax = 0.18 
            text = "" 
        intervals [2]:
            xmin = 0.18 
            xmax = 0.38 
            text = "we've" 
        intervals [3]:
            xmin = 0.38 
            xmax = 0.92 
            text = "expanded" 
        intervals [4]:
            xmin = 0.92 
            xmax = 1.04 
            text = "our" 
        intervals [5]:
            xmin = 1.04 
            xmax = 1.45 
            text = "support" 
        intervals [6]:
            xmin = 1.45 
            xmax = 1.55 
            text = "for" 
        intervals [7]:
            xmin = 1.55 
            xmax = 1.81 
            text = "civil" 
        intervals [8]:
            xmin = 1.8

## 2.1 Извлечение алайнмента

Мы получили начало и конец фонемы в секундах. Теперь нужно перевести их в фреймы мел-спектрограммы, а также округлить.

In [ ]:
def get_alignment(textgrid, sampling_rate, hop_length):
    phoneme_info = textgrid.tierDict['phones'].entryList
    phones = []
    durations = []

    for t in phoneme_info:
        if t.label == "":
            continue
        phones.append(t.label)
        start = t.start
        end = t.end
        ## YOUR CODE HERE
        duration_in_frames = <...>
        durations.append(
            duration_in_frames
        )
    return phones, durations


In [ ]:
from praatio import tgio
assert np.allclose(
    get_alignment(
        tgio.openTextgrid("tests/alignment.TextGrid", readRaw=True), 22050, 256
    )[1],
    np.load("tests/alignment.npy")
)

Теперь можно запустить препроцессинг датасета.

In [ ]:
from praatio import tgio
import pyworld as pw
import data


def extract_features(speaker, base_path, basename):
    tg_path = base_path / "TextGrid" / speaker / f"{basename}.TextGrid"
    wav_path = base_path / "split" / speaker / f"{basename}.wav"
    if not os.path.exists(tg_path):
        print("WARNING: no textgrid for basename", basename)
        return False
    

    textgrid = tgio.openTextgrid(tg_path, readRaw=True)
    phone, duration = get_alignment(
        textgrid,
        data_cfg.sampling_rate,
        data_cfg.hop_length
    )

    wav, _ = librosa.load(str(wav_path))

    mel, energy = data.extract_mel_energy(wav, sum(duration), data_cfg)
    f0 = data.extract_f0(wav, sum(duration), data_cfg)

    features = {
        'phone': phone,
        'energy': energy,
        'mel': mel,
        'f0': f0,
        'alignment': duration,
    }
    
    for key, feat in features.items():
        if feat is None:
            return False
        np.save(base_path / key / f"{key}-{basename}.npy", feat)
    return True

In [ ]:
for feature_type in ['phone', 'energy', 'mel', 'f0', 'alignment']:
    os.makedirs(base_path / feature_type, exist_ok=True)

for spk in os.listdir(split_path):
    spk_path = split_path / spk
    for filename in tqdm(os.listdir(spk_path)):
        if not filename.endswith(".wav"):
            continue
        basename = filename[:-4]
        extract_features(spk, base_path, basename)

## 2.2 Смотрим на ненормализованные фичи

f0 и energy лежат в директориях `{base_path}/f0` и `{base_path}/energy`.

Визуализируйте несколько примеров f0 и energy. Послушайте соответствующие записи, соотнесите скачки интонации со скачками фичей. Можно ли угадать интонацию предложения, посмотрев только на f0 и energy?

In [ ]:
<YOUR CODE HERE>

Скопируем данные VCTK в наш датасет:

In [ ]:
for feature_type in ['phone', 'energy', 'mel', 'f0', 'alignment']:
    for fn in tqdm(os.listdir(vctk_path / feature_type)):
        shutil.copy(vctk_path / feature_type / fn, base_path / feature_type)

## 2.3 Нормализация фичей

f0 и energy модель будет учить через MSE, при этом в этих фичах довольно много выбросов. Нормализуйте фичи, удалите выбросы.

In [ ]:
<YOUR CODE HERE>

Следующая ячейка построит статистики по получившемуся датасету: модели потребуются эти статистики, чтобы предсказывать f0 и energy.

In [ ]:
import data
import json

train_filenames, dev_filenames, speakers = data.train_test_split(base_path)
feature_stat = data.build_feature_stat(base_path, train_filenames)

with open(base_path / "speakers.json", "w") as f:
    json.dump({v: k for k, v in enumerate(speakers)}, f)

## 3.1 Реализация модели

Ниже приведена реализация FastSpeech2 для единственного спикера.

Но в нашей модели помимо фичей на вход модели будет подаваться эмбеддинг, соответствующий спикеру. Добавьте в модель этот эмбеддинг.

In [ ]:
data_cfg.update(feature_stat)
model_cfg = OmegaConf.load("conf/model.yaml")

In [ ]:
import os
import json

import torch
import torch.nn as nn
import torch.nn.functional as F


import torch
import torch.nn as nn
import torch.nn.functional as F

from transformer.Models import Encoder, Decoder
from transformer.Layers import PostNet
from utils import get_mask_from_lengths
from modules import VarianceAdaptor


class FastSpeech2(nn.Module):
    """ FastSpeech2 """

    def __init__(self, speaker_num, mel_channels, cfg, variance_cfg):
        super().__init__()

        self.encoder = Encoder(cfg)
        self.speaker_embedding = <...>
        self.variance_adaptor = VarianceAdaptor(
            cfg,
            f0_min=variance_cfg.f0_min,
            f0_max=variance_cfg.f0_max,
            energy_min=variance_cfg.energy_min,
            energy_max=variance_cfg.energy_max,
        )
        self.decoder = Decoder(cfg)
        self.mel_linear = nn.Linear(cfg.decoder_hidden, mel_channels)
        self.postnet = PostNet(postnet_kernel_size=5)

    def forward(
        self,
        src_seq,
        src_len,
        mel_len=None,
        d_target=None,
        p_target=None,
        e_target=None,
        max_src_len=None,
        max_mel_len=None,
        speaker=None, # [BATCH_SIZE]
        speaker_embedding=None, # [BATCH_SIZE, SPEAKER_EMBEDDING_SIZE] (need this if we want to infer with an embedding not present in training data)
    ):
        src_mask = get_mask_from_lengths(src_len, max_src_len)
        mel_mask = (
            get_mask_from_lengths(mel_len, max_mel_len) if mel_len is not None else None
        )

        encoder_output = self.encoder(src_seq, src_mask)
        if speaker is not None:
            if speaker_embedding is None:
                <...>
            encoder_output = <...>

        (
            variance_adaptor_output,
            d_prediction,
            d_rounded,
            p_prediction,
            e_prediction,
            mel_len,
            mel_mask,
        ) = self.variance_adaptor(
            encoder_output,
            encoder_output,
            src_mask,
            mel_mask,
            d_target,
            p_target,
            e_target,
            max_mel_len,
        )

        decoder_output = self.decoder(variance_adaptor_output, mel_mask)
        mel_output = self.mel_linear(decoder_output)
        mel_output_postnet = self.postnet(mel_output) + mel_output

        return (
            mel_output,
            mel_output_postnet,
            d_prediction,
            d_rounded,
            p_prediction,
            e_prediction,
            src_mask,
            mel_mask,
            mel_len,
        )
    
speaker_num = <...>
model = FastSpeech2(
    speaker_num, data_cfg.n_mel_channels, model_cfg, data_cfg
)

## 3.2 Обучение модели

Задание все еще про аналитику, так что обучение модели можно не писать:)

Тем не менее, стоит посмотреть на графики обучения (они пишутся в директорию logs в загружаемом в tensorboard формате), а также на примеры, которое модель синтезирует каждые 1000 итераций. Останавливайте обучение, когда качество достигнет приемлемого - не раньше через 50 тысяч итераций, но лучше - 100 тысяч. На VCTK разумного качества получится достигнуть быстрее. Для большинства следующих пунктов можно пользоваться инференсом только на примерах из VCTK.

Рекомендуем заглянуть в конфиг и настроить сохранение почаще, если работаете в колабе. Можно будет указать cfg.restore_step и продолжить обучение с нужного места.

In [ ]:
from train import train

train_cfg = OmegaConf.load("conf/train.yaml")
train(train_cfg, model_cfg, data_cfg, base_path, model)

## 4 Анализ модели

Пора инферить модель. Что можно делать:

1. Инферить тестовую выборку с gt-фичами (ground-truth) и predicted-фичами. Насколько помогают gt-фичи качеству звука? А естественности речи?
1. Найти несколько ошибок в длинах фонем. Посмотреть на обучающие данные - какие закономерности в них могли привести к ошибкам?
1. Изменить интонацию в синтезируемых примерах за счет ручного редактирования фичей. Как различаются предсказания фичей у разных спикеров?
1. Визуализировать эмбеддинги спикеров. Что можно сказать про пространство? Действительно ли эмбеддинги похожих спикеров близки друг к другу?
1. Сделать синтез с эмбеддингом отсутствовавшего в исходной выборке спикера: например, среднего между мужским и женским голосом (пользуйтесь параметром speaker_embedding). В каких ситуациях получается стабильный голос?
1. Синтезировать примеры Стива Джобса из двух его выступлений с gt-фичами и predicted-фичами. Одинаковое ли качество звука на двух выступлениях? Могут ли влиять фичи на качество звука?
1. Воспользоваться другим предобученным вокодером. На каких спикерах удалось улучшить качество?
1. Дообучить модель на примерах своего голоса. Получилось ли похоже?

За каждый пункт будет ставиться 2 балла: 1 балл за код и полученные аудио, 1 балл за выводы. В сумме до 8 баллов, с учетом бонусов - до 12.

### Hints:


2 способа перевода текста в фонемы:
1. (правильный) G2P модель от MFA (см. первый вызов MFA в этом ноутбуке)
2. (не совсем правильный, но простой) Через библиотеку g2p_en: для большинства случаев этот способ хорош, в некоторых кейсах фонемы будут отличаться от выхода MFA (а значит, и от обучающей выборки).

    ```
    from g2p_en import G2p

    g2p = G2p()
    
    phonemes = g2p(word)
    ```

Небольшая документация про то, какие параметры передавать на вход инференсу:

```
speaker_ids = id for each item in batch
texts = list of np.array[str] of phonemes
mel_len = None or frame count for each sample (take it from original mel lengths)
Ds = None or list of np.arrays[PHONEME_COUNT] with phoneme lengths. Sum should be equal to mel_len
f0s = None or list of np.array[FRAME_COUNT]. Do not use it when alignment is None
energies = None or list of np.array[FRAME_COUNT]. Do not use it when alignment is None
```

In [ ]:
ckpt = torch.load("ckpt/checkpoint_60000.pth.tar")
model = nn.DataParallel(model).cuda()
model.load_state_dict(ckpt['model'])

In [ ]:
basename = "obama_1_100s"
speaker = "obama"

speaker_ids = [speakers[speaker]]
texts = [np.load(base_path / "phone" / f"phone-{basename}.npy")]
f0s = [np.load(base_path / "f0" / f"f0-{basename}.npy")]
Ds = [np.load(base_path / "alignment" / f"alignment-{basename}.npy")]
mel_len = [Ds[0].sum()]
energies = None

In [ ]:
import soundfile as sf
import synthesise

<YOUR CODE HERE>

wavs, mels, Ds, f0s, energies = synthesise.synthesize(
    model,
    data_cfg,
    speaker_ids,
    texts,
    mel_len,
    Ds,
    f0s,
    energies,
)

for wav in wavs[:10]:
    display(Audio(wav, rate=22050))
    sf.write(f"output/{task_id}_{spk}_{basename}.wav", wav, 22050)

In [ ]:
<YOUR EXPERIMENTS HERE>

В LMS присылайте архив с решением: вложите туда ноутбук и примеры синтеза, которые вы анализировали. Начинайте названия файлов с номера пункта задания.